# Experiment of adding constant into genotype

Very simple target function:

In [41]:
def target_function(X):
    return 4.55 + X

import numpy as np

x_train = np.array([1,2,3,4,5])
y_train = target_function(x_train)

# First, let's try classic CGP, with constant as an input

In [52]:
%%time

x_train_c = np.c_[np.ones(len(x_train)), x_train]

funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.subtract, 2)

params = tengp.Parameters(2, 1, 1, 10, funset, real_valued=False)

from sklearn.metrics import mean_squared_error

res = tengp.simple_es(x_train_c, y_train, mean_squared_error, params, evaluations=100000)


CPU times: user 15 s, sys: 15.6 ms, total: 15 s
Wall time: 15.3 s


In [49]:
res[0]

Program, f:2.202499999999999

# Now PSO with constants in genotype

In [53]:
%%time

def constant(x, y):
    return x


cfunset = tengp.FunctionSet()
cfunset.add(constant, 2)
cfunset.add(np.add, 2)
cfunset.add(np.subtract, 2)

pso_params = tengp.Parameters(1, 1, 1, 10, cfunset, real_valued=True, constants=10)

import pygmo as pg

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        return self.bounds



bounds = tengp.individual.IndividualBuilder(pso_params).create().bounds

prob = pg.problem(cost_function(x_train.reshape(-1, 1), y_train, pso_params, bounds))

algo = pg.algorithm(pg.pso(gen=2000))

algo.set_verbosity(1)

pop = pg.population(prob, 50)

pop = algo.evolve(pop)

CPU times: user 27.4 s, sys: 46.9 ms, total: 27.5 s
Wall time: 28.1 s


In [51]:
print(pop.champion_f[0])

0.0
